In [1]:
using BenchmarkTools
using LinearAlgebra
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation

LinearAlgebraicRepresentation

In [2]:
square = Lar.cuboid([1,1])

([0.0 0.0 1.0 1.0; 0.0 1.0 0.0 1.0], [[1, 2, 3, 4]])

In [3]:
table = Lar.apply(Lar.t(-0.5,-0.5),square)

([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]])

In [4]:
structure = Lar.Struct([repeat([table,Lar.r(pi/2)],outer=2)...])

LinearAlgebraicRepresentation.Struct(Any[([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0], ([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4]]), [6.123233995736766e-17 -1.0 0.0; 1.0 6.123233995736766e-17 0.0; 0.0 0.0 1.0]], [[-0.5; -0.5], [0.5; 0.5]], "10386279109356727272", 2, "feature")

In [5]:
structure.box

2-element Array{Array{Float64,2},1}:
 [-0.5; -0.5]
 [0.5; 0.5]

## Vecchia funzione

In [38]:
function struct2lar(structure)
	listOfModels = Lar.evalStruct(structure)
	vertDict = Dict()
	index,defaultValue = 0,0
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Number,1}[] for k=1:m]

	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end

	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 2 methods)

In [39]:
@btime struct2lar(structure)

  32.899 μs (142 allocations: 8.77 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4], [3, 1, 4, 2]])

In [40]:
@code_warntype struct2lar(structure)

Variables
  #self#::Core.Compiler.Const(struct2lar, false)
  structure::LinearAlgebraicRepresentation.Struct
  @_3::Int64
  #36::var"#36#37"
  larmodel::Any
  W::Array{Array{Float64,1},1}
  V::Any
  chains::Any

Body::Tuple{Any,Vararg{Any,N} where N}
1 ─ %1  = Base.getproperty(Main.Lar, :evalStruct)::Any
│   %2  = (%1)(structure)::Any
│   %3  = Main.flatten(%2)::Tuple{Any,Array{Array{Float64,1},1}}
│   %4  = Base.indexed_iterate(%3, 1)::Core.Compiler.PartialStruct(Tuple{Any,Int64}, Any[Any, Core.Compiler.Const(2, false)])
│         (larmodel = Core.getfield(%4, 1))
│         (@_3 = Core.getfield(%4, 2))
│   %7  = Base.indexed_iterate(%3, 2, @_3::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Array{Array{Float64,1},1},Int64}, Any[Array{Array{Float64,1},1}, Core.Compiler.Const(3, false)])
│         (W = Core.getfield(%7, 1))
│   %9  = Base.getindex(larmodel, 1)::Any
│         Main.append!(%9, W)
│   %11 = Base.getindex(larmodel, 1)::Any
│         (V = Core._apply_itera

## Nuove funzioni

In [50]:
function flatten(listOfModels)
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Number,1}[] for k=1:m]
	vertDict = Dict()
	index,defaultValue = 0,0
	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end
	return larmodel,W
end

flatten (generic function with 1 method)

In [52]:
function struct2lar(structure) 
	larmodel,W = flatten(Lar.evalStruct(structure))
	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 2 methods)

In [43]:
@btime struct2lar(structure)

  33.099 μs (142 allocations: 8.77 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4], [3, 1, 4, 2]])

In [44]:
@benchmark struct2lar(structure)

BenchmarkTools.Trial: 
  memory estimate:  8.77 KiB
  allocs estimate:  142
  --------------
  minimum time:     33.300 μs (0.00% GC)
  median time:      37.999 μs (0.00% GC)
  mean time:        42.455 μs (3.06% GC)
  maximum time:     6.710 ms (97.70% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [45]:
@code_warntype struct2lar(structure)

Variables
  #self#::Core.Compiler.Const(struct2lar, false)
  structure::LinearAlgebraicRepresentation.Struct
  @_3::Int64
  #36::var"#36#37"
  larmodel::Any
  W::Array{Array{Float64,1},1}
  V::Any
  chains::Any

Body::Tuple{Any,Vararg{Any,N} where N}
1 ─ %1  = Base.getproperty(Main.Lar, :evalStruct)::Any
│   %2  = (%1)(structure)::Any
│   %3  = Main.flatten(%2)::Tuple{Any,Array{Array{Float64,1},1}}
│   %4  = Base.indexed_iterate(%3, 1)::Core.Compiler.PartialStruct(Tuple{Any,Int64}, Any[Any, Core.Compiler.Const(2, false)])
│         (larmodel = Core.getfield(%4, 1))
│         (@_3 = Core.getfield(%4, 2))
│   %7  = Base.indexed_iterate(%3, 2, @_3::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Array{Array{Float64,1},1},Int64}, Any[Array{Array{Float64,1},1}, Core.Compiler.Const(3, false)])
│         (W = Core.getfield(%7, 1))
│   %9  = Base.getindex(larmodel, 1)::Any
│         Main.append!(%9, W)
│   %11 = Base.getindex(larmodel, 1)::Any
│         (V = Core._apply_itera

## Modifiche

In [63]:
function flatten(listOfModels)
	W = Array{Float64,1}[]
	m = length(listOfModels[1])
	larmodel = [Array{Float64,1}[] for k=1:m]
	vertDict = Dict()
	index,defaultValue = 0,0
	for model in listOfModels
		V = model[1]
		for k=2:m
			for incell in model[k]
				outcell=[]
				for v in incell
					key = map(Lar.approxVal(7), V[:,v])
					if get(vertDict,key,defaultValue)==defaultValue
						index += 1
						vertDict[key]=index
						push!(outcell,index)
						push!(W,key)
					else
						push!(outcell,vertDict[key])
					end
				end
				append!(larmodel[k],[outcell])
			end
		end
	end
	return larmodel,W
end

flatten (generic function with 1 method)

In [64]:
function struct2lar(structure::Lar.Struct) 
	larmodel,W = flatten(Lar.evalStruct(structure))
	append!(larmodel[1], W)
	V = hcat(larmodel[1]...)
	chains = [convert(Lar.Cells, chain) for chain in larmodel[2:end]]
	return (V, chains...)
end

struct2lar (generic function with 2 methods)

In [65]:
@code_warntype struct2lar(structure)

Variables
  #self#::Core.Compiler.Const(struct2lar, false)
  structure::LinearAlgebraicRepresentation.Struct
  @_3::Int64
  #69::var"#69#70"
  larmodel::Any
  W::Array{Array{Float64,1},1}
  V::Any
  chains::Any

Body::Tuple{Any,Vararg{Any,N} where N}
1 ─ %1  = Base.getproperty(Main.Lar, :evalStruct)::Any
│   %2  = Base.getproperty(Main.Lar, :Struct)::Any
│   %3  = Core.typeassert(structure, %2)::LinearAlgebraicRepresentation.Struct
│   %4  = (%1)(%3)::Any
│   %5  = Main.flatten(%4)::Tuple{Any,Array{Array{Float64,1},1}}
│   %6  = Base.indexed_iterate(%5, 1)::Core.Compiler.PartialStruct(Tuple{Any,Int64}, Any[Any, Core.Compiler.Const(2, false)])
│         (larmodel = Core.getfield(%6, 1))
│         (@_3 = Core.getfield(%6, 2))
│   %9  = Base.indexed_iterate(%5, 2, @_3::Core.Compiler.Const(2, false))::Core.Compiler.PartialStruct(Tuple{Array{Array{Float64,1},1},Int64}, Any[Array{Array{Float64,1},1}, Core.Compiler.Const(3, false)])
│         (W = Core.getfield(%9, 1))
│   %11 = Base.getindex

In [66]:
@btime struct2lar(structure)

  31.401 μs (127 allocations: 8.09 KiB)


([-0.5 -0.5 0.5 0.5; -0.5 0.5 -0.5 0.5], [[1, 2, 3, 4], [3, 1, 4, 2]])